In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval

# Análise Exploratória de Dados Imobiliários

## 01. Conjunto de Dados : Contexto


Escrever uma breve descrição sobre os dados e algumas hipóteses

In [2]:
pd.set_option('display.max_columns', 30)

In [3]:
df = pd.read_csv('dados_imoveis_sp.csv')
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
0,"['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...",['101'],2574084550,[1],"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",[],[2],['111'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
1,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",['140'],2583748663,[2],"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",[2],[4],[],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
2,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",['50'],2562971980,[1],"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",[0],[1],['50'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
3,"['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...",['58'],2580478200,[1],"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",[],[1],[],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
4,"['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...",['64'],2583729583,[1],"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",[],[1],['80'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


## 02. Limpeza e Tratamento de dados

Os tipos de dados em todas as colunas deste dataset estão no formato **string**. Os dados numéricos precisam de uma **conversão de tipo**.

In [4]:
cols = ['usableAreas','parkingSpaces','suites','bathrooms','totalAreas','bedrooms']

for var in cols:
    s_extracted_digits = df[var].str.extract('\[[\']{0,1}(\d*)[\']{0,1}\]').squeeze()
    df[var] = s_extracted_digits.apply(lambda x: int(x) if x.isdigit() else np.nan)
    
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
0,"['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...",101,2574084550,1.0,"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",NaN,2,111.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
1,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",140,2583748663,2.0,"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",2.0,4,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
2,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",50,2562971980,1.0,"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",0.0,1,50.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
3,"['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...",58,2580478200,1.0,"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",NaN,1,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
4,"['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...",64,2583729583,1.0,"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",NaN,1,80.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


Diversas colunas também possuem bastante informações que nos ajudaram a melhor caracterizar cada imóvel.
No entanto, essas informações estão em formato bruto, "raw data", e precisam de um tratamento.

A coluna *'address'* possui características que lembram um dicionário. Mas seu tipo é string. 
Precisamos tratar essas linhas para navegar e extrair seus atributos com facilidade.

In [5]:
print(df['address'][0],'\n')
print('Tipo de dado:',type(df['address'][0]))

{'country': 'BR', 'zipCode': '04734003', 'geoJson': '', 'city': 'São Paulo', 'streetNumber': '1850', 'level': 'STREET', 'precision': 'ROOFTOP', 'confidence': 'VALID_STREET', 'stateAcronym': 'SP', 'source': 'CORREIOS', 'point': {'lon': -46.695829, 'source': 'GOOGLE', 'lat': -23.638282}, 'ibgeCityId': '', 'zone': 'Zona Sul', 'street': 'Avenida Adolfo Pinheiro', 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Santo Amaro', 'district': '', 'name': '', 'state': 'São Paulo', 'neighborhood': 'Santo Amaro', 'poisList': ['BS:Graham Bell C/B', 'BS:Graham Bell B/C', 'BS:Rua Verbo Divino, 61', 'BS:Américo Brasiliense C/B', 'BS:Parada Marechal Deodoro 2 - B/C', 'TS:Graham Bell C/B', 'TS:Graham Bell B/C', 'TS:Rua Verbo Divino, 61', 'TS:Américo Brasiliense C/B', 'TS:Parada Marechal Deodoro 2 - B/C', 'CS:7 Molinos', 'CS:Casa de Pães Neblina Paulista', 'CS:Casa de Bolo', 'CS:Berna', 'CS:Gêmel', 'VP:Kennel Club'], 'pois': [], 'valuableZones': [{'city': 'São Paulo', 'zone': 'Zona Sul', 'name': 'Cháca

O comando **literal_eval** é um interessante comando da biblioteca **ast – Abstract Syntax Trees**. 

Ele avalia uma string contendo uma expressão Python e a executa.

In [6]:
print(literal_eval(df['address'][0]),'\n')
print('Tipo de dado:',type(literal_eval(df['address'][0])))

{'country': 'BR', 'zipCode': '04734003', 'geoJson': '', 'city': 'São Paulo', 'streetNumber': '1850', 'level': 'STREET', 'precision': 'ROOFTOP', 'confidence': 'VALID_STREET', 'stateAcronym': 'SP', 'source': 'CORREIOS', 'point': {'lon': -46.695829, 'source': 'GOOGLE', 'lat': -23.638282}, 'ibgeCityId': '', 'zone': 'Zona Sul', 'street': 'Avenida Adolfo Pinheiro', 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Santo Amaro', 'district': '', 'name': '', 'state': 'São Paulo', 'neighborhood': 'Santo Amaro', 'poisList': ['BS:Graham Bell C/B', 'BS:Graham Bell B/C', 'BS:Rua Verbo Divino, 61', 'BS:Américo Brasiliense C/B', 'BS:Parada Marechal Deodoro 2 - B/C', 'TS:Graham Bell C/B', 'TS:Graham Bell B/C', 'TS:Rua Verbo Divino, 61', 'TS:Américo Brasiliense C/B', 'TS:Parada Marechal Deodoro 2 - B/C', 'CS:7 Molinos', 'CS:Casa de Pães Neblina Paulista', 'CS:Casa de Bolo', 'CS:Berna', 'CS:Gêmel', 'VP:Kennel Club'], 'pois': [], 'valuableZones': [{'city': 'São Paulo', 'zone': 'Zona Sul', 'name': 'Cháca

A coluna *'pricingInfos'* pode possuir dois dicionários. Um contendo preço de aluguel, e outro com preço de compra.
Estamos interessados em pegar apenas o preço de aluguel.

In [7]:
literal_eval(df['pricingInfos'][91])

[{'rentalInfo': {'period': 'MONTHLY',
   'warranties': ['INSURANCE_GUARANTEE', 'GUARANTOR']},
  'yearlyIptu': '120',
  'price': '410000',
  'businessType': 'SALE',
  'monthlyCondoFee': '700'},
 {'rentalInfo': {'period': 'MONTHLY',
   'warranties': ['INSURANCE_GUARANTEE', 'GUARANTOR'],
   'monthlyRentalTotalPrice': '2400'},
  'yearlyIptu': '120',
  'price': '1700',
  'businessType': 'RENTAL',
  'monthlyCondoFee': '700'}]

A coluna *'amenities'* apresenta as facilidades que cada imóvel pode oferecer.
- Como podemos contar a frequência para nosso conjunto dados?

In [8]:
df['amenities'][:5]

0    ['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...
1    ['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...
2    ['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...
3    ['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...
4    ['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...
Name: amenities, dtype: object

In [9]:
def extract_neighborhood(address):
    address = literal_eval(address)
    neighborhood = address['neighborhood']
    
    return neighborhood

In [10]:
def extract_zone(address):
    address = literal_eval(address)
    zone = address['zone']
    
    return zone

In [11]:
def extract_zipcode(address):
    address = literal_eval(address)
    zipCode = address['zipCode']
    
    return zipCode

In [12]:
def get_rental_price(pricingInfos):
    price = [info['price'] for info\
     in literal_eval(pricingInfos)\
     if info['businessType'] == 'RENTAL'][0]
    
    return float(price)

In [13]:
def strings_para_lista(string):
    lista = string.replace(' ','').replace("'","")
    lista = lista.replace('[','').replace(']','').split(',')
    
    return lista

In [14]:
df['amenities'] = df['amenities'].apply(strings_para_lista)

In [15]:
print('Top 10 Facilidades dos Imóveis de São Paulo:')
print(df['amenities'].explode().value_counts()[:10])

Top 10 Facilidades dos Imóveis de São Paulo:
ELEVATOR           5129
POOL               4613
PARTY_HALL         4110
BARBECUE_GRILL     3961
SERVICE_AREA       3729
GYM                3687
PLAYGROUND         3357
GARDEN             3203
INTERCOM           2899
GATED_COMMUNITY    2854
Name: amenities, dtype: int64


In [16]:
top10_amenities = list(df['amenities'].explode().value_counts()[:10].index)

In [17]:
def has_amenity(amenities,amenity):
    if amenity in amenities:
        return 1
    else:
        return 0

In [18]:
# Cria uma coluna para cada uma das 10 principais facilidades
# com valor binário, representando ausência ou ocorrência da mesma
for amenity in top10_amenities:
    df[amenity.lower()] = df['amenities'].apply(has_amenity,amenity=amenity)

In [19]:
df['zipCode'] = df['address'].apply(extract_zipcode)
df['zone'] = df['address'].apply(extract_zone)
df['neighborhood'] = df['address'].apply(extract_neighborhood)

In [20]:
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood
0,"[PETS_ALLOWED, ELEVATOR, GARDEN, ELECTRONIC_GA...",101,2574084550,1.0,"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",NaN,2,111.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,0,0,0,0,0,0,1,0,0,04734003,Zona Sul,Santo Amaro
1,"[POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GY...",140,2583748663,2.0,"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",2.0,4,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,1,1,1,0,1,1,1,0,0,01307000,Centro,Consolação
2,"[POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GA...",50,2562971980,1.0,"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",0.0,1,50.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,1,1,1,0,1,1,1,0,1,01209010,Centro,Santa Efigênia
3,"[POOL, BARBECUE_GRILL, GATED_COMMUNITY, GYM, G...",58,2580478200,1.0,"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",NaN,1,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",0,1,1,1,0,1,1,1,0,1,01127000,Centro,Bom Retiro
4,"[PETS_ALLOWED, GATED_COMMUNITY, ELECTRONIC_GAT...",64,2583729583,1.0,"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",NaN,1,80.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,0,0,0,0,0,0,0,0,1,05435001,Zona Oeste,Sumarezinho


In [21]:
df['rental_price'] = df['pricingInfos'].apply(get_rental_price)

In [22]:
df.drop(['amenities','address','pricingInfos'], axis=1,inplace=True)

In [23]:
df.head()

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood,rental_price
0,101,2574084550,1.0,NaN,2,111.0,2,1,0,0,0,0,0,0,1,0,0,04734003,Zona Sul,Santo Amaro,2300.0
1,140,2583748663,2.0,2.0,4,NaN,2,1,1,1,1,0,1,1,1,0,0,01307000,Centro,Consolação,9500.0
2,50,2562971980,1.0,0.0,1,50.0,2,1,1,1,1,0,1,1,1,0,1,01209010,Centro,Santa Efigênia,3000.0
3,58,2580478200,1.0,NaN,1,NaN,2,0,1,1,1,0,1,1,1,0,1,01127000,Centro,Bom Retiro,1900.0
4,64,2583729583,1.0,NaN,1,80.0,2,1,0,0,0,0,0,0,0,0,1,05435001,Zona Oeste,Sumarezinho,2400.0


Existem 'id' repetidos. Isso pode indicar repetição de padrões na base. De fato, existem no máximo 72 repetições na base

In [31]:
df['id'].unique().shape

(9928,)

Mas a função duplicated mostra que não existe nenhuma linha com informações duplicadas. Isso quer dizer que, embora existam 'id' duplicados as informações entre eles não são completamente semelhantes. Por isso, é necessário investigar mais a fundo.

In [53]:
df_a = df[df.duplicated()]
df_a

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood,rental_price
1500,61,2586722059,1.0,0.0,1,61.0,3,1,0,0,1,1,0,0,0,1,1,05564200,Zona Oeste,Jardim Esmeralda,2000.0
1800,160,2499389704,3.0,3.0,4,220.0,3,1,1,1,1,0,1,1,1,0,1,04660000,Zona Sul,Jardim Marajoara,9500.0
2000,79,2578244497,1.0,NaN,2,79.0,2,0,0,0,0,0,0,0,0,0,0,01156000,Zona Oeste,Barra Funda,2000.0
2500,75,2546649018,2.0,1.0,2,75.0,3,0,1,1,0,1,1,1,0,0,0,04294000,Zona Sul,Saúde,1800.0
3100,55,2586950643,0.0,0.0,1,88.0,2,0,0,0,0,0,0,0,0,0,0,04119060,Zona Sul,Vila Mariana,1800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9500,55,2585634009,1.0,0.0,1,55.0,2,0,1,0,1,0,0,0,0,0,0,04776901,Zona Sul,Socorro,2450.0
9600,161,2575411489,4.0,3.0,3,161.0,3,0,0,0,0,0,0,0,0,0,0,04513030,Zona Sul,Vila Nova Conceição,10000.0
9601,64,2537346780,1.0,0.0,2,NaN,2,1,1,0,1,0,0,1,1,0,0,04149120,Zona Sul,Jardim da Saúde,1600.0
9900,100,2588578730,2.0,1.0,2,NaN,2,1,1,1,1,1,1,0,1,1,1,01421000,Zona Oeste,Jardim Paulista,7500.0


In [54]:
df_b = df[df['id'].duplicated()]
df_b

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood,rental_price
1500,61,2586722059,1.0,0.0,1,61.0,3,1,0,0,1,1,0,0,0,1,1,05564200,Zona Oeste,Jardim Esmeralda,2000.0
1800,160,2499389704,3.0,3.0,4,220.0,3,1,1,1,1,0,1,1,1,0,1,04660000,Zona Sul,Jardim Marajoara,9500.0
2000,79,2578244497,1.0,NaN,2,79.0,2,0,0,0,0,0,0,0,0,0,0,01156000,Zona Oeste,Barra Funda,2000.0
2500,75,2546649018,2.0,1.0,2,75.0,3,0,1,1,0,1,1,1,0,0,0,04294000,Zona Sul,Saúde,1800.0
3100,55,2586950643,0.0,0.0,1,88.0,2,0,0,0,0,0,0,0,0,0,0,04119060,Zona Sul,Vila Mariana,1800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9500,55,2585634009,1.0,0.0,1,55.0,2,0,1,0,1,0,0,0,0,0,0,04776901,Zona Sul,Socorro,2450.0
9600,161,2575411489,4.0,3.0,3,161.0,3,0,0,0,0,0,0,0,0,0,0,04513030,Zona Sul,Vila Nova Conceição,10000.0
9601,64,2537346780,1.0,0.0,2,NaN,2,1,1,0,1,0,0,1,1,0,0,04149120,Zona Sul,Jardim da Saúde,1600.0
9900,100,2588578730,2.0,1.0,2,NaN,2,1,1,1,1,1,1,0,1,1,1,01421000,Zona Oeste,Jardim Paulista,7500.0


In [55]:
df_a['id'] == df_b['id']

ValueError: Can only compare identically-labeled Series objects

# Tratamento de Outliers

In [29]:
df.describe()

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,rental_price
count,10000.00000,1.000000e+04,9603.000000,8484.000000,10000.000000,8061.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.00000,10000.0000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,83.89140,2.576484e+09,1.323545,0.857732,1.931400,99.347600,2.084300,0.512900,0.461300,0.41100,0.39610,0.3729,0.368700,0.335700,0.320300,0.289900,0.285400,4482.165800
std,71.64535,5.292020e+07,1.114453,0.983135,1.190225,170.107159,0.839204,0.499859,0.498525,0.49204,0.48911,0.4836,0.482477,0.472258,0.466615,0.453738,0.451627,10503.741205
min,10.00000,6.506009e+07,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,500.000000
25%,49.00000,2.574096e+09,1.000000,0.000000,1.000000,50.000000,1.000000,0.000000,0.000000,0.00000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,1800.000000
50%,65.00000,2.582125e+09,1.000000,1.000000,2.000000,70.000000,2.000000,1.000000,0.000000,0.00000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,2760.000000
75%,96.00000,2.587231e+09,2.000000,1.000000,2.000000,103.000000,3.000000,1.000000,1.000000,1.00000,1.00000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,4500.000000
max,3300.00000,2.591836e+09,45.000000,20.000000,10.000000,6000.000000,7.000000,1.000000,1.000000,1.00000,1.00000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,710000.000000


# Tratamento de valores faltantes

Pelas informações do dataframe, existem 3 colunas com dados faltantes que precisam ser tratados.

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   usableAreas      10000 non-null  int64  
 1   id               10000 non-null  int64  
 2   parkingSpaces    9603 non-null   float64
 3   suites           8484 non-null   float64
 4   bathrooms        10000 non-null  int64  
 5   totalAreas       8061 non-null   float64
 6   bedrooms         10000 non-null  int64  
 7   elevator         10000 non-null  int64  
 8   pool             10000 non-null  int64  
 9   party_hall       10000 non-null  int64  
 10  barbecue_grill   10000 non-null  int64  
 11  service_area     10000 non-null  int64  
 12  gym              10000 non-null  int64  
 13  playground       10000 non-null  int64  
 14  garden           10000 non-null  int64  
 15  intercom         10000 non-null  int64  
 16  gated_community  10000 non-null  int64  
 17  zipCode      

Separando as colunas que contem dados faltantes em um dataframe

In [27]:
df_nan_cols = df[['parkingSpaces','suites','totalAreas']]

parkingSpaces    0.0397
suites           0.1516
totalAreas       0.1939
dtype: float64

Obtendo as porcentagens de dados faltantes com relação a quantidade total em cada coluna

In [28]:
df_nan_cols.isnull().sum()/len(df_nan_cols)*100

parkingSpaces     3.97
suites           15.16
totalAreas       19.39
dtype: float64

## Análise dos dados

### Análise univariada

### Análise bivariada

### Análise de correlação